In [39]:
from bs4 import BeautifulSoup, SoupStrainer
import httplib2
import re
import pandas as pd
from itertools import chain
import time 

In [52]:
scp_main_df = pd.DataFrame(columns=['code', 'title', 'text', 'link', 'rating'], index=None)

In [42]:
http = httplib2.Http()

In [ ]:
status, response = http.request('https://scp-wiki.wikidot.com/scp-series')
soup = BeautifulSoup(response, 'html.parser')

# note from 2235 onwards there are a couple that do not have the standard codes in directory pages, will keep the standard code for now
# may go back and adjust 

# the page links all follow pattern and can loop through nums to string just zero padded basically
for n in range(2235,7000):
  # timeout to limit load on the site
  if n % 240 == 0:
    print(n)
    time.sleep(60)
  num_as_str = '{}{}'.format('0' * (3 - len(str(n))), str(n)) if n < 1000 else str(n)

  url = 'https://scp-wiki.wikidot.com/scp-{}'.format(num_as_str)
  status, response = http.request(url)
  soup = BeautifulSoup(response, 'html.parser')
  text = [i for i in soup.find(id = 'page-content').find_all('p')]
  # p tags include things like image captions may want to exclude, can do with scp-image-caption class there and check other cases
  for i in text:
    if hasattr(i.parent, 'class'):
      if i.parent.get("class") == 'scp-image-caption':
        text.remove(i)
  text = [i.text for i in text]
  rating = soup.find(class_ = 'number prw54353')
  if rating == None:
    pass
  else:
    rating = rating.text
  # below needs to accoun for past 999
  if n < 1000:
    status, response = http.request('https://scp-wiki.wikidot.com/scp-series')
  else:
    status, response = http.request('https://scp-wiki.wikidot.com/scp-series-{}'.format(str(int(n//1000)+1)))
  soup = BeautifulSoup(response, 'html.parser')
  if soup.find('a', text='SCP-{}'.format(num_as_str)) == None:
    # can try next li after previous num find? assuming there won't be any consecutive non standard titles...
    title = soup.find('a', text='SCP-{}'.format(str(n-1))).parent.find_next_sibling().text.split(" - ")[-1]
    # print(title)
    # and here can also change/note the other code? if later want to add another column for these edge cases 
    # eg the ozymandias effect is both scp2235 and ura9611
  else:
    title = soup.find('a', text='SCP-{}'.format(num_as_str)).parent.text.split(" - ")[-1]
  
  # print(rating)
  # quotes to preserve commas and not cause csv issues
  info = ['SCP-{}'.format(num_as_str), "\"{}\"".format(title), "\"{}\"".format(text), url, rating]
  scp_main_df.loc[len(scp_main_df)] = info



In [48]:
scp_main_df.head

<bound method NDFrame.head of           code  ... rating
0      SCP-001  ...   None
1      SCP-002  ...  +1697
2      SCP-003  ...   +760
3      SCP-004  ...  +1091
4      SCP-005  ...   +640
...        ...  ...    ...
2229  SCP-2230  ...   +114
2230  SCP-2231  ...    +56
2231  SCP-2232  ...   +319
2232  SCP-2233  ...    +72
2233  SCP-2234  ...   +114

[2234 rows x 5 columns]>

In [49]:
scp_main_df.to_csv('scp6999.csv', mode='a', header=False)

In [ ]:
pd.read_csv('scp6999.csv', delimiter=',', quotechar='"')